In [ ]:
import os
import json
import asyncio
from typing import List, Dict, Any
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_text_splitters import RecursiveCharacterTextSplitter

os.environ["NO_PROXY"] = "192.168.1.10,localhost,127.0.0.1"

# ================= 配置部分 =================
#API_KEY = os.environ.get("DASHSCOPE_API_KEY", "sk-")
#API_KEY = os.environ.get("GEMINI_API_KEY", "")
API_KEY="ollama"
#MODEL_URL="https://dashscope.aliyuncs.com/compatible-mode/v1"
#MODEL_URL="https://generativelanguage.googleapis.com/v1beta/openai/"
MODEL_URL="http://192.168.1.10:11434/v1" 
ALLOWED_PATH = "/Users/lichao/Downloads/mcp_workspace"
INPUT_FILE = "input.md"
OUTPUT_FILE = "output_zh.md"
#MODEL_NAME = "qwen-plus"
#MODEL_NAME = "gemini-2.5-flash"
MODEL_NAME = "qwen3:latest"

#client = OpenAI(api_key=DASHSCOPE_API_KEY, base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"

# ================= 1. 内存上下文 (Memory Context) =================
class MemoryContext:
    def __init__(self):
        self.raw_content = ""
        self.source_chunks: List[str] = []
        self.read_cursor: int = 0
        self.output_buffer: List[str] = []

    def get_full_output(self) -> str:
        return "\n\n".join(self.output_buffer)

memory = MemoryContext()

# ================= 2. 本地实现的 4 个中间工具 =================

def local_tool_split():
    """[本地] 拆分文本到内存"""
    print(f"⚙️ [本地能力] 正在拆分文本...")
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)
    chunks = splitter.split_text(memory.raw_content)
    memory.source_chunks = chunks
    memory.read_cursor = 0
    memory.output_buffer = []
    #print(f"拆分完成，内存中已有 {len(chunks)} 个片段。")
    return f"拆分完成，内存中已有 {len(chunks)} 个片段。"

def local_tool_get_next_and_translate():
    """[本地] 取下一段, 并进行翻译"""
    if memory.read_cursor >= len(memory.source_chunks) or memory.read_cursor > 10:
        return "EOF"
    chunk = memory.source_chunks[memory.read_cursor]
    idx = memory.read_cursor
    memory.read_cursor += 1
    #print(f"📄 [本地能力] 取出第 {idx+1}/{len(memory.source_chunks)} 段")

    work_client = OpenAI(api_key=API_KEY, base_url=MODEL_URL)
    resp = work_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "直译 Markdown 为中文，保留格式。"},
            {"role": "user", "content": chunk}
        ],
        temperature=0.1 # 翻译要求低温度
    )
    print(f"📄 [本地能力] 翻译： {idx+1}/{len(memory.source_chunks)} 段，{resp.choices[0].message.content[:100]}")
    return resp.choices[0].message.content

def local_tool_save(content: str):
    """[本地] 保存片段到内存"""
    memory.output_buffer.append(content)
    print(f"💾 [本地能力] 结果已存入内存 (共 {len(memory.output_buffer)} 段)")
    return "已保存到内存 buffer。"

# 定义这 4 个本地工具的 Schema (OpenAI 格式)
LOCAL_TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "split_text_to_memory",
            "description": "将刚才读入到内存中的长文本拆分并存入内存队列。无参数",
            "parameters": {"type": "object", "properties": {}, "required": []}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_chunk_translate_segment",
            "description": "从内存队列中取出下一个文本片段进行翻译。如果返回 'EOF' 表示取完了。",
            "parameters": {"type": "object", "properties": {}}
        }
    }
]

# ================= 3. 主逻辑 =================

async def run_dynamic_agent():
    # 1. 连接 MCP 服务器
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-filesystem", ALLOWED_PATH],
    )
    
    print("🔌 连接 MCP 服务器中...")
    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # ==========================================
            # 关键步骤 A: 动态获取 MCP 工具列表
            # ==========================================
            mcp_list_result = await session.list_tools()
            mcp_tools_map = {t.name: t for t in mcp_list_result.tools} # 用于后续查找
            
            print(f"🌍 发现 MCP 服务器提供了 {len(mcp_tools_map)} 个工具: {list(mcp_tools_map.keys())}")
            
            # 将 MCP 工具转换为 OpenAI 格式
            openai_mcp_tools = []
            for tool in mcp_list_result.tools:
                openai_mcp_tools.append({
                    "type": "function",
                    "function": {
                        "name": tool.name,
                        "description": tool.description,
                        "parameters": tool.inputSchema # MCP Schema 兼容 OpenAI parameters
                    }
                })

            # ==========================================
            # 关键步骤 B: 合并工具列表 (4个本地 + N个远程)
            # ==========================================
            all_tools = openai_mcp_tools + LOCAL_TOOLS_SCHEMA

            # ==========================================
            # 关键步骤 C: 启动 Agent 循环
            # ==========================================
            client = OpenAI(api_key=API_KEY, base_url = MODEL_URL)

            messages = [
                {
                    "role": "system", 
                    "content": (
                            "你是一个严谨的 Markdown 翻译 Agent。**将英文Markdonw转成中文Markdown**"
                            f"你的工作目录根路径是：{ALLOWED_PATH}\n"  # <--- 加上这一句
                            "请注意：调用 read_file 和 write_file 时，**必须使用完整的绝对路径**。\n"
                            f"例如：读写文件时请使用 {os.path.join(ALLOWED_PATH, INPUT_FILE)}\n\n"
                            "请严格按照以下顺序执行，**禁止跳步**，**禁止并发调用工具**：\n\n"
                            "1. 第一步：调用 `read_text_file` 读取文件。当前允许操作的根目录是: {ALLOWED_PATH}\n"
                            "   🛑 **执行完这一步后，必须停止思考，等待工具返回真实的文件内容。**\n"
                            "   🚫 **绝对不要**在不知道文件真实内容的情况下，就去调用 `split_text_to_memory`，不要臆造文本内容！\n\n"
                            "2. 第二步：当看到工具返回的文件内容后，调用 `split_text_to_memory` (本地) 拆分内容。\n"
                            "3. 第三步：循环调用 `get_chunk_translate_segment`。\n"
                            "4. 第四步：收到**EOF**消息后，使用 `write_file` 写入文件，content 参数填 '__USE_MEMORY_BUFFER__'。当前允许操作的根目录是: {ALLOWED_PATH}"
                            "5. 特别注意：**回复时不要带思维链**"
                    )
                },
                {"role": "user", "content": f"请读取 {INPUT_FILE}，翻译后保存到 {OUTPUT_FILE}"}
            ]

            print("🚀 Agent 开始工作...")

            should_prune_history = False
            while True:
                # 思考
                #print(f"send LLM message:{messages}")
                response = client.chat.completions.create(
                    model=MODEL_NAME, messages=messages, tools=all_tools, tool_choice="auto"
                )
                msg = response.choices[0].message
                messages.append(msg)

                if not msg.tool_calls:

                    print(f"💬 Agent: {msg, msg.content}")
                    print(f"⚠️出错了！！！，强制退出Agent")
                    break

                # 【强制串行】如果发现 Agent 一次性想做两件事（比如既读又拆），只允许它做第一件
                if len(msg.tool_calls) > 1:
                    print(f"⚠️ 警告：检测到并发调用 {len(msg.tool_calls)} 个工具，强制截断，只执行第一个。")
                    # 只保留第一个工具调用，删除其他的，防止它用幻觉参数执行第二个工具
                    msg.tool_calls = [msg.tool_calls[0]]

                # 执行
                for tool_call in msg.tool_calls:
                    name = tool_call.function.name
                    #print(f"DEBUG - Raw Arguments: {tool_call.function.arguments!r}")
                    args = json.loads(tool_call.function.arguments)
                    tool_result = ""
                    
                    print(f"👉 决策: {name},{args}")

                    # --- 路由判断逻辑 ---
                    
                    # 情况 1: 如果是 MCP 提供的工具 (动态发现的)
                    if name in mcp_tools_map:
                        # 特殊拦截：如果是写入操作，且使用了占位符，则注入内存数据
                        if name == "write_file" and args.get("content") == "__USE_MEMORY_BUFFER__":
                            print("⚡️ [系统] 拦截到写入请求，正在注入内存 Buffer...")
                            args["content"] = memory.get_full_output()
                        
                        # 调用远程 MCP
                        try:
                            res = await session.call_tool(name, args)
                            # 简化返回给 LLM 的内容，节省 Token (特别是 read_file 返回大文本时)
                            if name == "read_file" or name == "read_text_file":
                                tool_result = res.content[0].text # 必须返回真实文本给 LLM，因为它要传给 split
                                if(tool_result is not None):
                                    print(f"read text file context:{tool_result[:100]}, file len:{len(tool_result)}")
                                    # 2. 【核心】存入内存，不给 LLM 看！
                                    memory.raw_content = tool_result
                                
                                    # 3. 构造给 LLM 看的“假”结果 (简报)
                                    tool_result = f"文件读取成功！内容已存入系统内存。总长度: {len(tool_result)} 字符。请继续调用 split_text_to_memory。"
                                
                                    #print(f"✅ [系统拦截] 文件内容已截获并存入内存，未泄露给 LLM。")
                                    
                                else:
                                    print(f"🚫严重错误，未读到任何内容...")
                            else:
                                tool_result = "执行成功 (MCP Tool)"
                                break
                        except Exception as e:
                            tool_result = f"MCP Error: {e}"
                            break

                    # 情况 2: 如果是本地工具
                    elif name == "split_text_to_memory":
                        tool_result = local_tool_split()
                        # 为了省 Token，这里可以欺骗 LLM 说文本已加载，不要把结果返给它
                        tool_result = "[已省略的大文本]" 

                        # 拆分完也可以清理一次
                        should_prune_history = True
                        status_summary = (
                            f"文件已拆分为 {len(memory.source_chunks)} 段。\n"
                            f"👉 请开始循环：调用 `get_next_chunk_from_memory` 获取第 {memory.read_cursor} 段。"
                        )

                    elif name == "get_chunk_translate_segment":
                        tool_result = local_tool_get_next_and_translate()
                        #if(tool_result is not None):
                            #print(f"get next chunk:{tool_result[:100]}...")
                        if tool_result != "EOF":
                            tool_result = local_tool_save(tool_result)
                            # 标记：这一轮闭环结束了，该清理垃圾了
                            should_prune_history = True
                            
                            # 构造一个状态简报，告诉 LLM 下一步该干嘛，防止它失忆
                            # 我们可以从 memory 对象里获取当前进度
                            current = memory.read_cursor
                            total = len(memory.source_chunks)
                            status_summary = (
                                f"✅ 第 {current} 段翻译已保存 (进度: {current}/{total})。\n"
                                "历史上下文已清理以释放内存。\n"
                                "👉 请立即调用 `get_chunk_translate_segment` 继续下一段。"
                            )
                    else:
                        tool_result = f"Error: 未知工具 {name}"

                    # 反馈结果
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(tool_result)})

                    if should_prune_history:
                        #print(f"🧹 [系统] 检测到闭环完成，正在清理上下文 (保留 System Prompt)...")
                        
                        # 1. 保留 System Prompt (messages[0])
                        system_prompt = messages[0]
                        first_user_prompt = messages[1]
                        
                        # 2. 构造一个新的 User 消息，作为“接力棒”
                        # 这非常重要！因为清空历史后，LLM 需要知道自己现在的状态
                        new_user_message = {
                            "role": "user", 
                            "content": f"系统通知：{status_summary}"
                        }
                        
                        # 3. 重置 messages 列表
                        messages = [system_prompt, first_user_prompt, new_user_message]
                        
                        # 这样，下一次循环时，发送给 LLM 的只有 2 条消息，
                        # Token 消耗瞬间回到最低水平！
                        should_prune_history = False

if __name__ == "__main__":
    # Jupyter 环境兼容
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        loop = None

    if loop and loop.is_running():
        await run_dynamic_agent()
    else:
        asyncio.run(run_dynamic_agent())

🔌 连接 MCP 服务器中...
🌍 发现 MCP 服务器提供了 14 个工具: ['read_file', 'read_text_file', 'read_media_file', 'read_multiple_files', 'write_file', 'edit_file', 'create_directory', 'list_directory', 'list_directory_with_sizes', 'directory_tree', 'move_file', 'search_files', 'get_file_info', 'list_allowed_directories']
🚀 Agent 开始工作...
👉 决策: read_text_file,{'path': '/Users/lichao/Downloads/mcp_workspace/input.md'}
read text file context:# Agentic Design Patterns

*A Hands-On Guide to Building Intelligent Systems* *1* *, * *Antonio Gull, file len:861242
👉 决策: split_text_to_memory,{}
⚙️ [本地能力] 正在拆分文本...
👉 决策: get_chunk_translate_segment,{}
📄 [本地能力] 翻译： 1/1469 段，# Agentic Design Patterns

*一本实践指南：构建智能系统* *1* *, * *Antonio Gulli* * * $$ 目录 - 总共424页   = 1+2+1+1+4
💾 [本地能力] 结果已存入内存 (共 1 段)
👉 决策: get_chunk_translate_segment,{}
📄 [本地能力] 翻译： 2/1469 段，1. 第1章：提示链（代码），12页 [最终版，最后阅读完成，代码无误]  
2. 第2章：路由（代码），13页 [最终版，最后阅读完成，代码无误]  
3. 第3章：并行化（代码），15页 [最终版
💾 [本地能力] 结果已存入内存 (共 2 段)
👉 决策: get_chunk_translate_segment,{}
📄 [本地